In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import math

In [2]:
#retrieving the data
parkingdata = pd.read_csv("parking_duration_of_parking_event_vs_street_ID.csv")
parkingdata = parkingdata.sample(n=10000)
parkingdata.size

130000

In [3]:
#Converting all the feature attributes to uppercase for uniformity

parkingdata['Area Name'] = parkingdata['Area Name'].str.upper()
parkingdata['Street Name'] = parkingdata['Street Name'].str.upper()
parkingdata['Between Street 1'] = parkingdata['Between Street 1'].str.upper()
parkingdata['Between Street 2'] = parkingdata['Between Street 2'].str.upper()
parkingdata['Street Marker'] = parkingdata['Street Marker'].str.upper()
parkingdata['Sign'] = parkingdata['Sign'].str.upper()

parkingdata.head(5)

,Area Name,Street Name,Between Street 1,Between Street 2,Side Of Street,Street Marker,Arrival Time,Departure Time,Duration of Parking Event (in seconds),Sign,In Violation?,Street ID,Device ID
408919,COUNTY,LONSDALE STREET,KING STREET,WILLIAM STREET,4,3203S,13/10/2011 07:35:52 AM,13/10/2011 07:37:44 AM,112,CW TOW M-F 7:00-9:30,1,894,1908
1487374,COURTNEY,ELIZABETH STREET,PELHAM STREET,QUEENSBERRY STREET,5,4727W,11/11/2011 10:59:28 AM,11/11/2011 11:03:00 AM,212,2P TKT A M-F 7:30-18:30,0,627,3498
1584273,COURTNEY,QUEENSBERRY STREET,O'CONNELL STREET,ELIZABETH STREET,4,7667S,14/11/2011 12:45:14 PM,14/11/2011 01:26:24 PM,2470,1/4P M-F 7:30-18:30,1,1175,4062
7657152,MAGISTRATES,LA TROBE STREET,KING STREET,WILLIAM STREET,3,3924N,16/05/2012 02:51:27 PM,16/05/2012 02:53:19 PM,112,2P TKT A M-SAT 7:30-20:30,0,856,2294
1664674,CITY SQUARE,COLLINS STREET,SWANSTON STREET,RUSSELL STREET,3,1948N,16/11/2011 12:13:53 PM,16/11/2011 12:30:44 PM,1011,1/2P M-SAT 7:30-19:30,0,528,1099


In [4]:
#Removing all redundant extra whitespaces
for x in parkingdata.columns:
    if parkingdata[x].dtype == object:
        parkingdata[x] = parkingdata[x].str.strip()

In [5]:
#converting object to datetime
parkingdata['Arrival Time'] = pd.to_datetime(parkingdata['Arrival Time'])
parkingdata['Departure Time'] = pd.to_datetime(parkingdata['Departure Time'])

In [6]:
#Target field identification

target = parkingdata['Street Name']

unique_vals = target.unique()
target.replace(to_replace = unique_vals, value = list(range(len(unique_vals))), inplace = True)

target.head(10)

408919      0
1487374     1
1584273     2
7657152     3
1664674     4
11738171    5
1888090     6
8648324     5
11356795    4
5071207     2
Name: Street Name, dtype: int64

In [7]:
#Preliminary Target Features identification (Pre-Hill Climbing)

#Convert possible targets to dtype = int
intDf = parkingdata.copy()
#Drop useless fields
intDf.drop('Arrival Time', axis = 1, inplace = True)
intDf.drop('Departure Time', axis = 1, inplace = True)
intDf.drop('Street Marker', axis = 1, inplace = True)
#Drop target field
intDf.drop('Street Name', axis = 1, inplace = True)
#Drop already int fields (Will add back after)
intDf.drop('In Violation?', axis = 1, inplace = True)
intDf.drop('Side Of Street', axis = 1, inplace = True)
intDf.drop('Duration of Parking Event (in seconds)', axis = 1, inplace = True)
intDf.drop('Street ID', axis = 1, inplace = True)
intDf.drop('Device ID', axis = 1, inplace = True)

#Transform the remainder non-int fields' values to unique int identifiers
for column in intDf:
    unique_vals = intDf[column].unique()
    intDf[column].replace(to_replace = unique_vals, value = list(range(len(unique_vals))), inplace = True)

#Create dataframe for all the relevant features
features = intDf.copy()

#Add back int fields (If not the target field)
features['In Violation?'] = parkingdata['In Violation?']
features['Side Of Street'] = parkingdata['Side Of Street']
features['Duration of Parking Event (in seconds)'] = parkingdata['Duration of Parking Event (in seconds)']
features['Street ID'] = parkingdata['Street ID']
features['Device ID'] = parkingdata['Device ID']
#Add back datetime features with only the meaningful subset of data
features['Arrival Hour'] = parkingdata['Arrival Time'].dt.hour
features['Departure Hour'] = parkingdata['Departure Time'].dt.hour

#Table view of features dataframe
features.head(10)

,Area Name,Between Street 1,Between Street 2,Sign,In Violation?,Side Of Street,Duration of Parking Event (in seconds),Street ID,Device ID,Arrival Hour,Departure Hour
408919,0,0,0,0,1,4,112,894,1908,7,7
1487374,1,1,1,1,0,5,212,627,3498,10,11
1584273,1,2,2,2,1,4,2470,1175,4062,12,13
7657152,2,0,0,3,0,3,112,856,2294,14,14
1664674,3,3,3,4,0,3,1011,528,1099,12,12
11738171,4,4,4,5,0,1,6947,681,2901,11,12
1888090,5,5,5,6,0,5,483,839,877,14,14
8648324,6,6,6,7,1,1,4668,681,2811,12,13
11356795,7,7,4,8,0,4,820,528,6761,20,20
5071207,8,8,7,9,0,4,912,1175,4148,11,11


In [8]:
#Parameter Tuning

#Define the parameters to tune and the values to tune to
params_randomforest = [
                { 
                   'n_estimators' : (10,20,30,50,70,90,100),
                    'criterion' : ('gini', 'entropy'),
                    'max_depth' : (3, 5, 7, 9, 10, 15),
                    'max_features' : ('auto', 'sqrt'),
                    'min_samples_split' : (2, 4, 6, 8, 10, 14, 16)
                    
                }
            ]

In [9]:
cv_method = RepeatedStratifiedKFold(n_splits = 5, 
                                    n_repeats = 1, 
                                    random_state = 1)

gs_randomforest = GridSearchCV(RandomForestClassifier(), 
                      param_grid = params_randomforest, 
                      cv = cv_method,
                      verbose = True,
                      scoring = 'accuracy',
                      n_jobs = -1,
                      return_train_score = True)

In [10]:
#Fit the model with the dataset
bestModel = gs_randomforest.fit(features, target)

Fitting 5 folds for each of 1176 candidates, totalling 5880 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   54.1s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 5880 out of 5880 | elapsed: 14

In [11]:
gs_randomforest.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_split': 6,
 'n_estimators': 100}

In [12]:
#Find the best values for the  parameters of the model (Formatted output)
best_e = bestModel.best_estimator_.get_params()['n_estimators']
best_C = bestModel.best_estimator_.get_params()['criterion']
best_maxd =  bestModel.best_estimator_.get_params()['max_depth']
best_maxf =  bestModel.best_estimator_.get_params()['max_features']
best_mins =  bestModel.best_estimator_.get_params()['min_samples_split']

print('Best estimator:', best_e)
print('Best criterion:', best_C)
print('Best max depth:', best_maxd)
print('Best max features:', best_maxf)
print('Best min_samples_split:', best_mins)

Best estimator: 100
Best criterion: entropy
Best max depth: 10
Best max features: sqrt
Best min_samples_split: 6


In [13]:
#Visualise the parameter configurations from fitting the model with the dataset
results_randomforest = pd.DataFrame(gs_randomforest.cv_results_['params'])
results_randomforest['test_score'] = gs_randomforest.cv_results_['mean_test_score']
results_randomforest

,criterion,max_depth,max_features,min_samples_split,n_estimators,test_score
0,gini,3,auto,2,10,0.5026
1,gini,3,auto,2,20,0.5143
2,gini,3,auto,2,30,0.5254
3,gini,3,auto,2,50,0.5373
4,gini,3,auto,2,70,0.5439
...,...,...,...,...,...,...
1171,entropy,15,sqrt,16,30,0.9983
1172,entropy,15,sqrt,16,50,0.9985
1173,entropy,15,sqrt,16,70,0.9988
1174,entropy,15,sqrt,16,90,0.9987


In [14]:
#Re-Define model with the optimal parameter values AFTER HILL CLIMBING
randomforest = metric = RandomForestClassifier(n_estimators = best_e, 
                               criterion = best_C, 
                               max_depth = best_maxd, 
                               max_features = best_maxf,
                               min_samples_split = best_mins                
                               )

In [15]:
# Hill climbing w/ SVM
new_Ind = []
cur_MaxScore = 0.0
col_num = len(features.columns)
col_Ind_Random = shuffle(range(0, col_num), random_state = 1)
features_array = features.values

for cur_f in range(col_num):
    new_Ind.append(col_Ind_Random[cur_f])
    newData = features_array[:, new_Ind]
    x_train, x_test, y_train, y_test = train_test_split(newData, target, test_size=0.2, random_state=1)
    fit = randomforest.fit(x_train, y_train)
    cur_Score = randomforest.score(x_test, y_test)
    
    if cur_Score < cur_MaxScore:
        new_Ind.remove(col_Ind_Random[cur_f])
    else:
        cur_MaxScore = cur_Score
        print ("Score with " + str(len(new_Ind)) + " selected features: " + str(cur_Score))
print("\nIndexs of the desired features")
print(new_Ind)

Score with 1 selected features: 0.3575
Score with 2 selected features: 0.7465
Score with 3 selected features: 0.7715
Score with 4 selected features: 0.8965
Score with 5 selected features: 1.0
Score with 6 selected features: 1.0
Score with 7 selected features: 1.0

Indexs of the desired features
[2, 3, 1, 0, 7, 10, 8]


In [16]:
features_hc = pd.DataFrame()
for index in new_Ind:
    colName = features.columns[index]
    features_hc[colName] = features[colName]
features_hc.head(10)

,Between Street 2,Sign,Between Street 1,Area Name,Street ID,Departure Hour,Device ID
408919,0,0,0,0,894,7,1908
1487374,1,1,1,1,627,11,3498
1584273,2,2,2,1,1175,13,4062
7657152,0,3,0,2,856,14,2294
1664674,3,4,3,3,528,12,1099
11738171,4,5,4,4,681,12,2901
1888090,5,6,5,5,839,14,877
8648324,6,7,6,6,681,13,2811
11356795,4,8,7,7,528,20,6761
5071207,7,9,8,8,1175,11,4148


In [17]:
bestModel = gs_randomforest.fit(features_hc, target)

Fitting 5 folds for each of 1176 candidates, totalling 5880 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   21.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   51.2s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 5880 out of 5880 | elapsed: 13

In [18]:
gs_randomforest.best_params_

{'criterion': 'gini',
 'max_depth': 15,
 'max_features': 'auto',
 'min_samples_split': 4,
 'n_estimators': 100}

In [19]:
best_e_hc = bestModel.best_estimator_.get_params()['n_estimators']
best_C_hc = bestModel.best_estimator_.get_params()['criterion']
best_maxd_hc =  bestModel.best_estimator_.get_params()['max_depth']
best_maxf_hc =  bestModel.best_estimator_.get_params()['max_features']
best_mins_hc =  bestModel.best_estimator_.get_params()['min_samples_split']

print('Best estimators:', best_e_hc)
print('Best criterion:', best_C_hc)
print('Best max depth:', best_maxd_hc)
print('Best max features:', best_maxf_hc)
print('Best min_samples_split:', best_mins_hc)

Best estimators: 100
Best criterion: gini
Best max depth: 15
Best max features: auto
Best min_samples_split: 4


In [20]:
#Visualise the parameter configurations from fitting the model with the dataset
results_randomforest = pd.DataFrame(gs_randomforest.cv_results_['params'])
results_randomforest['test_score'] = gs_randomforest.cv_results_['mean_test_score']
results_randomforest

,criterion,max_depth,max_features,min_samples_split,n_estimators,test_score
0,gini,3,auto,2,10,0.5102
1,gini,3,auto,2,20,0.5341
2,gini,3,auto,2,30,0.5162
3,gini,3,auto,2,50,0.5245
4,gini,3,auto,2,70,0.5370
...,...,...,...,...,...,...
1171,entropy,15,sqrt,16,30,0.9980
1172,entropy,15,sqrt,16,50,0.9992
1173,entropy,15,sqrt,16,70,0.9990
1174,entropy,15,sqrt,16,90,0.9989


In [21]:
randomforest = metric = RandomForestClassifier(n_estimators = best_e_hc, 
                               criterion = best_C_hc, 
                               max_depth = best_maxd_hc, 
                               max_features = best_maxf_hc,
                               min_samples_split = best_mins_hc                
                               )

In [22]:
#Defining training and testing groups
x_train, x_test, y_train, y_test = train_test_split(features_hc, target, test_size = 0.5, random_state = 4)

#Training the model previously defined
randomforest.fit(x_train, y_train)

#Obtaining and printing out results from the model (Confusion Matrix)
predicted = randomforest.predict(x_test)
cm = confusion_matrix(y_test,predicted)
print("Confusion Matrix")
print(cm)

#Printing the numerical result of the confusion matrix
print("\n[Train/test split] score: {:.5f}".format(randomforest.score(x_test, y_test)))

Confusion Matrix
[[469   0   0 ...   0   0   0]
 [  0 230   0 ...   0   0   0]
 [  0   0  53 ...   0   0   0]
 ...
 [  0   0   0 ...   7   0   0]
 [  0   0   0 ...   0   1   0]
 [  0   0   0 ...   0   0   2]]

[Train/test split] score: 0.99860


In [23]:
#Repeat with KFold - creating groups
kf = KFold(n_splits = 5, random_state = 4, shuffle = True)


#Repeat with KFold - Training model (previously defined) and obtaining its output
kFoldTotal = 0
for k, (train_index, test_index) in enumerate(kf.split(features_hc)):
    x_train, x_test = features_hc.iloc[train_index], features_hc.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    randomforest.fit(x_train, y_train)
    kFoldTotal += randomforest.score(x_test, y_test)
    print("[fold {0}] score: {1:.5f}".format(k, randomforest.score(x_test, y_test)))

#Printing out the results
roundedTotal = round(kFoldTotal/5, 5)
print("\nRandom forest mean score [5 folds] = " + str(roundedTotal))

[fold 0] score: 1.00000
[fold 1] score: 0.99950
[fold 2] score: 1.00000
[fold 3] score: 0.99950
[fold 4] score: 0.99800

Random forest mean score [5 folds] = 0.9994
